## Importation of libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Dense, SimpleRNN, Embedding
from tensorflow.keras.utils import to_categorical

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import copy as cp

# Input() is used to instantiate a Keras tensor.
# Dense implements the operation: output = activation(dot(input, kernel) + bias) 
# LSTM : Seq to Seq model 
# Model groups layers into an object with training and inference features 

# from keras.preprocessing.text import one_hot
# from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


## Data Visualization And Encoding

In [2]:
# Load the datasets

fname_train = "data/finnish-task1-train"
fname_test = "data/finnish-task1-test"
fname_dev = "data/finnish-task1-dev"

train = np.loadtxt(fname_train,dtype = str)
test = np.loadtxt(fname_test,dtype = str)
dev = np.loadtxt(fname_dev,dtype = str)

In [3]:
print(train)
print(train.shape)
print('-----------')
print(test)
print(test.shape)
print('-----------')
print(dev)
print(dev.shape)

[['Ã¤Ã¤kkÃ¶stÃ¤Ã¤' 'pos=V,polar=POS,mood=IMP,tense=PRS,per=3,num=SG'
  'Ã¤Ã¤kkÃ¶stÃ¤kÃ¶Ã¶n']
 ['Ã¤Ã¤kkÃ¶stÃ¤Ã¤' 'pos=V,voice=ACT,aspect=PROSP'
  'Ã¤Ã¤kkÃ¶stÃ¤mÃ¤isillÃ¤Ã¤n']
 ['aalloittaisuus' 'pos=N,case=ON+ESS,num=PL' 'aalloittaisuuksilla']
 ...
 ['zoonoosi' 'pos=N,case=PRIV,num=SG' 'zoonoositta']
 ['zsaari' 'pos=N,case=IN+LAT,num=PL' 'zsaareihin']
 ['zumbata' 'pos=V,polar=POS,mood=POT,tense=PRS,per=2,num=PL'
  'zumbannette']]
(12693, 3)
-----------
[['alkeiskoppi' 'pos=N,case=NOM,num=SG' 'alkeiskoppi']
 ['lenkkitossut' 'pos=N,case=ON+ESS,num=PL' 'lenkkitossuilla']
 ['baritonitorvi' 'pos=N,case=PRIV,num=SG' 'baritonitorvetta']
 ...
 ['katkeroida' 'pos=V,polar=POS,mood=IND,tense=PRS,per=3,num=SG'
  'katkeroi']
 ['paarmalintu' 'pos=N,case=TRANS,num=PL' 'paarmalinnuiksi']
 ['malisiÃ¶Ã¶si' 'pos=ADJ,case=IN+ABL,num=SG' 'malisiÃ¶Ã¶sistÃ¤']]
(23633, 3)
-----------
[['aakkosto' 'pos=N,case=NOM,num=PL' 'aakkostot']
 ['aallottaa' 'pos=V,mood=PURP,voice=ACT' 'aallottaakseen']
 ['aaltoluku' 'pos

In [4]:
input_train = [ i+','+j for i,j in zip(train[:,0],train[:,1])]
output_train = train[:,2]

print(input_train[0:10])

['Ã¤Ã¤kkÃ¶stÃ¤Ã¤,pos=V,polar=POS,mood=IMP,tense=PRS,per=3,num=SG', 'Ã¤Ã¤kkÃ¶stÃ¤Ã¤,pos=V,voice=ACT,aspect=PROSP', 'aalloittaisuus,pos=N,case=ON+ESS,num=PL', 'aallokas,pos=ADJ,case=FRML,num=PL', 'aalloppi,pos=N,case=IN+ESS,num=PL', 'aaltoilla,pos=V,polar=POS,mood=COND,tense=PRS,per=1,num=PL', 'aaltoilla,pos=V,polar=POS,mood=IMP,tense=PST,per=2,num=SG', 'aaltoilla,pos=V,polar=POS,mood=POT,tense=PRS,per=3,num=SG', 'aaltomuoto,pos=N,case=PRIV,num=PL', 'aaltopelti,pos=N,case=NOM,num=PL']


In [5]:
# Input
tokenizer_in = Tokenizer()
tokenizer_in.fit_on_texts(input_train)
encoded_in = tokenizer_in.texts_to_sequences(input_train)
word_index_in = tokenizer_in.word_index
vocabulary_in = list(word_index_in.keys())

In [ ]:
print("Vocabulary (Input)",vocabulary_in)
print("The word index (Input)",word_index_in)
print("Encoded version of the inputs (Input)",encoded_in)

In [7]:
#Output
tokenizer_out = Tokenizer()
tokenizer_out.fit_on_texts(output_train)
encoded_out = tokenizer_out.texts_to_sequences(output_train)
word_index_out = tokenizer_out.word_index
vocabulary_out = list(word_index_out.keys())

In [26]:
print("Vocabulary (Output)",vocabulary_out[0:10])
print("The word index (Output)",list(word_index_out.values())[0:10])
print("Encoded version of the outputs (Output)",encoded_out[0:10])


Vocabulary (Output) ['antilooppina', 'bio', 'aero', 'aktualisoinnin', 'beeta', 'karoteenit', 'oikeuksissa', 'data', 'erityisnosto', 'oikeuden']
The word index (Output) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Encoded version of the outputs (Output) [[42], [43], [44], [45], [46], [47], [48], [49], [50], [51]]


In [27]:
## Padding
def padding(seq):
  max_len = np.max([len(sentence) for sentence in seq])
  return pad_sequences(seq, maxlen = max_len, padding = 'post')


## Padding
def padding_2(seq,n):
  return pad_sequences(seq, maxlen = n, padding = 'post')

In [28]:
inputs = padding(encoded_in)
outputs = padding(encoded_out)

In [29]:
encoder_input_data = to_categorical(inputs, num_classes = len(vocabulary_in) + 1) # input
decoder_input_data = to_categorical(outputs, num_classes = len(vocabulary_out) + 1) # output

In [30]:
decoder_target_data = np.zeros(decoder_input_data.shape)
for index,data in enumerate(decoder_input_data) : 
    X = cp.deepcopy(data)
    decoder_target_data[index] = np.roll(X,-1,axis=0)

In [31]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(12693, 13, 9944)
(12693, 4, 12738)
(12693, 4, 12738)


In [32]:
def encode_data(data):
    
    inputs = [ i+','+j for i,j in zip(data[:,0],data[:,1])]
    outputs = data[:,2]
    
    # inputs
    I_tokenizer_in = Tokenizer()
    I_tokenizer_in.fit_on_texts(inputs)
    I_encoded_in = tokenizer_in.texts_to_sequences(inputs)
    I_word_index_in = tokenizer_in.word_index
    I_vocabulary_in = list(I_word_index_in.keys())
    
    
    # outputs   
    O_tokenizer_out = Tokenizer()
    O_tokenizer_out.fit_on_texts(outputs)
    O_encoded_out = tokenizer_out.texts_to_sequences(outputs)
    O_word_index_out = tokenizer_out.word_index
    O_vocabulary_out = list(O_word_index_out.keys())
    
    inputs = padding_2(I_encoded_in,13)
    outputs = padding_2(O_encoded_out,4)
    
    _encoder_input_data = to_categorical(inputs, num_classes = len(vocabulary_in) + 1) # input
    _decoder_input_data = to_categorical(outputs, num_classes = len(vocabulary_out) + 1) # output
    
    _decoder_target_data = np.zeros(_decoder_input_data.shape)
    for index,data in enumerate(_decoder_input_data): 
        X = cp.deepcopy(data)
        _decoder_target_data[index] = np.roll(X,-1,axis=0)
    
    return _encoder_input_data,_decoder_input_data,_decoder_target_data

In [93]:
# Encode test data
encoder_input_data_val,decoder_input_data_val,decoder_target_data_val = encode_data(test)

# Encode dev data
encoder_input_data_val_dev,decoder_input_data_val_dev,decoder_target_data_val_dev = encode_data(dev)

In [94]:
print(encoder_input_data_val_dev.shape)
print(decoder_input_data_val_dev.shape)
print(decoder_target_data_val_dev.shape)

(1598, 13, 9944)
(1598, 4, 12738)
(1598, 4, 12738)


In [34]:
print(encoder_input_data_val.shape)
print(decoder_input_data_val.shape)
print(decoder_target_data_val.shape)

(23633, 13, 9944)
(23633, 4, 12738)
(23633, 4, 12738)


In [35]:
"""
Encoder Architecture
"""

NUM_HIDDEN_UNITS = 256 # NUM_HIDDEN_LAYERS
BATCH_SIZE = 500
NUM_EPOCHS = 20

encoder_vocab_size = len(vocabulary_in) + 1
decoder_vocab_size = len(vocabulary_out) + 1

encoder_inputs = Input(shape=(None, encoder_vocab_size))
encoder_lstm = LSTM(units=NUM_HIDDEN_UNITS, return_state=True)
# x-axis: time-step lstm
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c] # We discard `encoder_outputs` and only keep the states.

In [36]:
"""
Decoder Architecture
"""
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_inputs = Input(shape=(None, decoder_vocab_size))
decoder_lstm = LSTM(units=NUM_HIDDEN_UNITS, return_sequences=True, return_state=True)
# x-axis: time-step lstm
decoder_outputs, de_state_h, de_state_c = decoder_lstm(decoder_inputs, initial_state=encoder_states) # Set up the decoder, using `encoder_states` as initial state.
decoder_softmax_layer = Dense(decoder_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [37]:
"""
Encoder-Decoder Architecture
"""
# Define the model that will turn, `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics = ['accuracy']) # Set up model

In [38]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 9944)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 12738)  0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 10445824    input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  13306880    input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [39]:
model.fit(x=[encoder_input_data, decoder_input_data], y=decoder_target_data,
          batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_split=0.2) # Run training

Train on 10154 samples, validate on 2539 samples
Epoch 1/20
10154/10154 [==============================] - 104s 10ms/step - loss: 4.6259 - accuracy: 0.7116 - val_loss: 3.1712 - val_accuracy: 0.7481
Epoch 2/20
10154/10154 [==============================] - 66s 6ms/step - loss: 2.8114 - accuracy: 0.7483 - val_loss: 3.0842 - val_accuracy: 0.7481
Epoch 3/20
10154/10154 [==============================] - 64s 6ms/step - loss: 2.5667 - accuracy: 0.7483 - val_loss: 3.0045 - val_accuracy: 0.7481
Epoch 4/20
10154/10154 [==============================] - 66s 7ms/step - loss: 2.4652 - accuracy: 0.7483 - val_loss: 3.0488 - val_accuracy: 0.7481
Epoch 5/20
10154/10154 [==============================] - 64s 6ms/step - loss: 2.4327 - accuracy: 0.7483 - val_loss: 3.1215 - val_accuracy: 0.7481
Epoch 6/20
10154/10154 [==============================] - 64s 6ms/step - loss: 2.3773 - accuracy: 0.7483 - val_loss: 3.1448 - val_accuracy: 0.7481
Epoch 7/20
10154/10154 [==============================] - 64s 6ms/s

In [96]:
# Evaluate the model on the dev data using `evaluate`
print("Evaluate dev data ...")
results = model.evaluate(x=[encoder_input_data_val_dev, decoder_input_data_val_dev], y=decoder_target_data_val_dev,
          batch_size=BATCH_SIZE)
print("dev loss, dev acc:", results)

Evaluate dev data ...
1598/1598 [==============================] - 5s 3ms/step
dev loss, dev acc: [1.9830278556546819, 0.9960888624191284]


Good result so we try on test dataset

In [40]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate test data ...")
results = model.evaluate(x=[encoder_input_data_val, decoder_input_data_val], y=decoder_target_data_val,
          batch_size=BATCH_SIZE)
print("test loss, test acc:", results)

Evaluate on test data ...
23633/23633 [==============================] - 86s 4ms/step
test loss, test acc: [1.9751797968175162, 0.9993970394134521]
